#BehaviorScoringRun.ipynb




##🪰 This notebook is the entry point for the **BehaviorScoring** pipeline.

---




It keeps
implementation details inside modules and streamlines the workflow: set up the
environment, select paths, run the analysis, and produce consistent scored outputs.The emphasis is a user-friendly, reproducible flow: clear separation of config from logic, minimal editing in one place, and version-pinned runs.

This notebook guides you through a standard sequence: configure once, stage inputs, run scoring, and sync outputs.

#### Run order
```
1) Setup scoring environment  
2) Import configs & stage to local mirrors  
3) Run pipeline  
```




#### Dependencies
```
BehaviorScoringRun.ipynb
├── BehaviorScoringMain.py
├── BehaviorScoringFunctions.py
├── BehaviorScoringConfig.py
├── BehaviorScoringColabConfig.py
├── ExperimentConfig.py
└── PathConfig.py
```

#PREPARE ENVIROMENT

In [1]:
# Pin core library versions and show environment details

# python==3.11.13
# Pin libs to versions compatible with Colab's preinstalled stack.
!pip install -q \
  numpy==2.0.2 \
  pandas==2.2.2 \
  scikit-learn==1.6.1 \
  scipy==1.14.1

import sys, platform, numpy, pandas, sklearn, scipy
print("Python:", sys.version.split()[0], "| Platform:", platform.platform())
print("NumPy :", numpy.__version__,
      "| pandas :", pandas.__version__,
      "| sklearn :", sklearn.__version__,
      "| SciPy :", scipy.__version__)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 18.0 MB/s eta 0:00:00
Python: 3.11.13 | Platform: Linux-6.1.123+-x86_64-with-glibc2.35
NumPy : 2.0.2 | pandas : 2.2.2 | sklearn : 1.6.1 | SciPy : 1.14.1


In [3]:
# Cell 1 — Mount + inject PathConfig (DEV/PROD) + make Config importable + imports (OG style)

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# ============================ EDIT ME (choose one) ==========================
USE_TEMP_PATHCONFIG = True  # True = DEV (inject root); False = PROD (load file)

# PROD: absolute path to your real PathConfig.py (with a real __EXP_ROOT__)
pPathConfig = "__PLACEHOLDER_PATHCONFIG_ROOT__"

# DEV: PathConfig.py to inject, and the temporary experimental root to use
TEMP_PATHCONFIG      = "/content/drive/MyDrive/Matheus_e_Rodrigo/TestRun/Protocols/Codes/Config/PathConfig.py"
TEMP_EXPERIMENT_ROOT = "/content/drive/MyDrive/Matheus_e_Rodrigo/TestRun"
# ===========================================================================

import sys, types, importlib.util, importlib
from pathlib import Path

# --- inject or load PathConfig as a module object ---
if USE_TEMP_PATHCONFIG:
    p_pathconfig = Path(TEMP_PATHCONFIG)
    exp_root     = Path(TEMP_EXPERIMENT_ROOT)
    assert p_pathconfig.exists(), f"TEMP PathConfig.py not found: {p_pathconfig}"
    assert exp_root.exists(),     f"TEMP experimental root not found: {exp_root}"

    code = p_pathconfig.read_text().replace('"__EXP_ROOT__"', f'"{exp_root.as_posix()}"')
    PathConfig = types.ModuleType("PathConfig")
    exec(compile(code, str(p_pathconfig), "exec"), PathConfig.__dict__)
    print("TEMP PathConfig injected ✓")
else:
    p_pathconfig = Path(pPathConfig)
    assert p_pathconfig.exists(), f"PathConfig.py not found: {p_pathconfig}"
    spec = importlib.util.spec_from_file_location("PathConfig", str(p_pathconfig))
    PathConfig = importlib.util.module_from_spec(spec)
    sys.modules["PathConfig"] = PathConfig
    spec.loader.exec_module(PathConfig)
    print("Production PathConfig loaded ✓")

# --- put Codes & subpackages on sys.path so 'Config' and code modules import cleanly ---
codes_dirs = [
    Path(PathConfig.pCodes),
    Path(PathConfig.pCodes) / "Config",
    Path(PathConfig.pCodes) / "BehaviorScoring",
    Path(PathConfig.pCodes) / "CreateDataFrames",
]
for d in codes_dirs:
    d_str = str(d)
    if d_str not in sys.path:
        sys.path.insert(0, d_str)

# --- preload 'Config' and satisfy 'from Config import PathConfig' pattern (OG style) ---
importlib.import_module("Config")
sys.modules["Config.PathConfig"] = PathConfig

# --- import config modules (as modules), scoring modules, and ColabConfig (as module) ---
from Config import ExperimentConfig          # module
from Config import BehaviorScoringConfig     # module
from Config import BehaviorScoringColabConfig as BSC  # module with helpers
import BehaviorScoringFunctions as BSF
import BehaviorScoringMain as BSM

import shutil, time, importlib

# sanity: show the root used in headers
print("PROCESSING ROOT:", PathConfig.pExperimentalRoot)


Mounted at /content/drive
TEMP PathConfig injected ✓
PROCESSING ROOT: /content/drive/MyDrive/Matheus_e_Rodrigo/TestRun


In [4]:
# Cell 2 — Load Drive paths & create local output mirrors (no input staging)

# Drive root + canonical Drive paths based on your PathConfig
drive_root, drive_paths = BSC.load_configs(PathConfig)  # (Path, DrivePaths)

# Create local mirrors under /content ONLY for outputs (no copying inputs)
local_paths = BSC.local_mirrors(drive_root, drive_paths, local_root=None, verbose=False)


In [5]:
# Cell 3 — Validate inputs on Drive (OG style instantiation from modules)

EXPconfig = ExperimentConfig
BSconfig  = BehaviorScoringConfig

# Validate tracked/pose inputs on Drive (does not stage/copy)
pose_scoring = BSC.validate_inputs(drive_paths, pose_scoring=EXPconfig.POSE_SCORING, verbose=False)

# Keep EXPconfig in sync with auto-detect (optional)
try:
    EXPconfig.POSE_SCORING = bool(pose_scoring)
except Exception:
    pass


In [6]:
import importlib, Config.BehaviorScoringColabConfig as BSC
importlib.reload(BSC)

print("ColabConfig file:", getattr(BSC, "__file__", "(module injected)"))
print("Has make_mixed_pathconfig?", hasattr(BSC, "make_mixed_pathconfig"))
print("Has print_finish?", hasattr(BSC, "print_finish"))
print("Has start_background_sync?", hasattr(BSC, "start_background_sync"))

ColabConfig file: /content/drive/MyDrive/Matheus_e_Rodrigo/TestRun/Protocols/Codes/Config/BehaviorScoringColabConfig.py
Has make_mixed_pathconfig? True
Has print_finish? True
Has start_background_sync? True


In [7]:
# Cell 4 — Build mixed PathConfig (read from Drive, write to local)

# Keep headers & PROCESSING line pointing to Drive (pExperimentalRoot),
# but write outputs (Scored/ScoredPose/ScoredError) to local mirrors for batching.
PATHconfig = BSC.make_mixed_pathconfig(PathConfig, drive_paths, local_paths)


In [8]:
# Cell 5 — Start background sync (batched saves)

# Batch size (files per push); tweak here to 10/30/50 as you like
BATCH_SIZE = 100

# Start background sync that uploads in batches quietly
BSC.start_background_sync(
    local_paths,
    drive_paths,
    pose_scoring=EXPconfig.POSE_SCORING,
    batch_size=BATCH_SIZE
)


#RUN

In [ ]:
# Cell 6 — Run the scorer (streams header, pinned bar, and session summary)

# Make sure we use the latest in-session defs
importlib.reload(BSF)
importlib.reload(BSM)

run_stats = BSM.behavior_scoring_main(
    PATHconfig=PATHconfig,
    EXPconfig=EXPconfig,
    BSconfig=BSconfig,
    BSF=BSF
)

# Stop watcher; do a final quiet push of remaining files
BSC.stop_background_sync()
BSC.sync_outputs_back(
    local_paths,
    drive_paths,
    pose_scoring=EXPconfig.POSE_SCORING,
    verbose=False
)

# Print the exact block you want
BSC.print_finish(
    dest_path=str(PathConfig.pExperimentalRoot),
    scoring_seconds=float(run_stats.get("scoring_seconds", 0.0))
)


============================= SCORING SESSION =============================


  PROCESSING    ------  /content/drive/MyDrive/Matheus_e_Rodrigo/TestRun
  POSE SCORING  --------------------------------------------------  True

  FILES FOUND  -----------------------------------------------------  60
  TO SCORE     -----------------------------------------------------  60
  SKIPPING     ------------------------------------------------------  0
               ------------------------------  scored: 0  ---  errors: 0




```text
  SCORING  [#######....................................................]
           ----------------  file 7/60  ---  4.78s/file  ---  04m13s eta
  SCORED   ----------------------------------------------------------  7
```